In [ ]:
import os
import cv2
import random
import shutil
from  tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/Smart Retail Checkout')

In [ ]:
dataset_path = 'Dataset'
items = ['Amul_Ice_Cream', 'Cake', 'CeraVe', 'HnS_Shampoo', 'Nestle_Milk_Powder', 'Plum', 'Thums_Up', 'Wild_Stone']

images = {}

for item in items:
    item_path = os.path.join(dataset_path, item)
    images[item] = []

    if os.path.exists(item_path):
        for filename in os.listdir(item_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(item_path, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    images[item].append(img)
                else:
                    print(f"Warning: Failed to load {img_path}")
    else:
        print(f"Warning: Directory {item_path} does not exist")

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.03,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=False,
    vertical_flip=False,
    brightness_range=[0.85, 1.15],
    fill_mode='nearest'
)

In [ ]:
for item in items:
    safe_name = item.replace(' ', '_')
    os.makedirs(f'Aug_Dataset/Train/{safe_name}', exist_ok=True)

In [ ]:
def augment_train(x, item_name):
    safe_name = item_name.replace(' ', '_')
    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=f'Aug_Dataset/Train/{safe_name}', save_prefix=safe_name, save_format='jpg'):
        i += 1
        if i >= 60:
            break

In [ ]:
for item in items:
    for img in images[item]:
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)
        augment_train(x, item)

In [ ]:
def move_random_images(source_dir, destination_dir, percentage=0.2):
    # Create the destination directory if it doesn't exist
    os.makedirs(destination_dir, exist_ok=True)

    # Get a list of all image files in the source directory
    all_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]
    image_files = [f for f in all_files if f.lower().endswith(('png', 'jpg', 'jpeg', 'gif', 'bmp', 'tiff'))]

    # Randomly select 20% of the images
    num_to_move = int(len(image_files) * percentage)
    images_to_move = random.sample(image_files, num_to_move)

    # Move the selected images to the destination directory
    for image in images_to_move:
        src_path = os.path.join(source_dir, image)
        dst_path = os.path.join(destination_dir, image)
        shutil.move(src_path, dst_path)

In [ ]:
for item in items:
    safe_name = item.replace(' ', '_')
    source_dir = f'Aug_Dataset/Train/{safe_name}'
    destination_dir = f'Aug_Dataset/Test/{safe_name}'
    move_random_images(source_dir, destination_dir, percentage=0.2)